In [3]:
from dotenv import load_dotenv
from langchain_teddynote import logging

load_dotenv()
logging.langsmith("CH12-RAG")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH12-RAG


In [6]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory


prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "당신은 Question-Answering 챗봇입니다. 주어진 질문에 대한 답변을 제공해주세요."
        ),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human","#Question:\n{question}"),
    ]
)

llm = ChatOpenAI()
chain = prompt | llm | StrOutputParser()
store = {}

def get_session_history(session_ids):
    print(f"[대화 세션ID]: {session_ids}")
    if session_ids not in store:
        store[session_ids] = ChatMessageHistory()
    return store[session_ids]

chain_with_history  = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="question",
    history_messages_key="chat_history"
)

chain_with_history.invoke(
    {"question": "나의 이름은 민호입니다."},
    config={"configurable": {"session_id": "abc123"}},
)



[대화 세션ID]: abc123


'알겠습니다. 민호님, 무엇을 도와드릴까요?'

In [7]:
chain_with_history.invoke(
    {"question":"내 이름이 뭐라고?"},
    config={"configurable":{"session_id": "abc123"}},
)

[대화 세션ID]: abc123


'당신의 이름은 민호입니다.'

## RAG + RunnableWithMessageHistory

In [13]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PDFPlumberLoader
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.output_parsers import StrOutputParser

from operator import itemgetter


loader = PDFPlumberLoader("국방백서.pdf")
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)

split_documents = text_splitter.split_documents(docs)

embeddings = OpenAIEmbeddings()

vectorstore = FAISS.from_documents(documents=split_documents, embedding=embeddings)

retriever = vectorstore.as_retriever()

prompt = PromptTemplate.from_template(
    """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the question. 
If you don't know the answer, just say that you don't know. 
Answer in Korean.

#Previous Chat History:
{chat_history}

#Question: 
{question} 

#Context: 
{context} 

#Answer:"""
)

llm = ChatOpenAI(model_name="gpt-4o",temperature=0)

chain = (
    {
        "context": itemgetter("question")|
        retriever,
        "question": itemgetter("question"),
        "chat_history": itemgetter
        ("chat_history"),
    }
    |prompt
    |llm
    |StrOutputParser()
)

In [14]:
store= {}

def get_session_history(session_ids):
    print(f"[대화 세션ID]: {session_ids}")
    if session_ids not in store:
        store[session_ids] = ChatMessageHistory()
    return store[session_ids]

rag_with_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="question",
    history_messages_key="chat_history",
)



In [15]:
rag_with_history.invoke(
    {"question": "대한민국 전력은?"},
    config={"configurable": {"session_id": "rag123"}}
)


[대화 세션ID]: rag123


'대한민국은 2023년부터 2027년까지 107.4조원을 투자하여 전방위 안보위협에 대비하고, 북한 핵·WMD 위협에 대응하기 위한 한국형 3축체계 핵심전력을 확충하고 있습니다. 또한, 첨단전력 중심의 전력구조로 개편을 추진하고 있으며, 사이버·테러, 재난 등 비전통적 위협에 능동적으로 대응하고 있습니다. 국방 R&D 및 국내 방위산업 활성화에도 투자를 확대하고 있습니다.'

In [16]:
rag_with_history.invoke(
    {"question":"이전 답변을 영어로 번역해주세요."},
    config={"configurable": {"session_id":"rag123"}}
)

[대화 세션ID]: rag123


"South Korea is investing 107.4 trillion won from 2023 to 2027 to prepare for all-round security threats and to expand key forces of the Korean three-axis system to respond to North Korea's nuclear and WMD threats. Additionally, South Korea is pushing to reorganize its force structure centered on advanced power and actively respond to non-traditional threats such as cyber, terrorism, and disasters. Investments are also being expanded in defense R&D and the revitalization of the domestic defense industry.\n\n대한민국은 2023년부터 2027년까지 107.4조원을 투자하여 전방위 안보위협에 대비하고, 북한 핵·WMD 위협에 대응하기 위한 한국형 3축체계 핵심전력을 확충하고 있습니다. 또한, 첨단전력 중심의 전력구조로 개편을 추진하고 있으며, 사이버·테러, 재난 등 비전통적 위협에 능동적으로 대응하고 있습니다. 국방 R&D 및 국내 방위산업 활성화에도 투자를 확대하고 있습니다."